In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/financial-performance-prediction/sample_submission.csv
/kaggle/input/financial-performance-prediction/data_dictionary.txt
/kaggle/input/financial-performance-prediction/train.csv
/kaggle/input/financial-performance-prediction/test.csv


## Importing Libraries

In [2]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, KFold, GridSearchCV, RandomizedSearchCV, cross_val_score
from category_encoders import TargetEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.impute import KNNImputer


from sklearn.linear_model import LinearRegression

from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

from sklearn.metrics import (r2_score, mean_absolute_percentage_error)
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

## Fucntions

In [3]:
def basic_info(df):

    """
    Takes a DataFrame as input, and gives the basic info like shape, missing values count, duplicated rows, unique values and dtypes of the features
    
    Args:
        df (pandas.DataFrame): The DataFrame for which you want the details of

    Returns: 
        None 
    """
    print(f"shape of the date : \n\trows = {df.shape[0]}, columns = {df.shape[1]}\n")
    missing_val_count = df.isna().sum().sum()
    print(f"missing values: \n\tcount = {missing_val_count}")
    if missing_val_count != 0:
        missing_data = df.isna().sum().reset_index().rename({"index" : "feature", 0 : "missing_val_count"}, axis = 1)
        missing_data =  missing_data[missing_data.missing_val_count > 0]
        missing_data["missing_val_percentage"] = np.round((missing_data["missing_val_count"] / df.shape[0]) * 100, 2)
        missing_data = missing_data.sort_values(by = "missing_val_count", ascending = False)
        display(missing_data)

    print(f"duplicated records: \n\tcount = {df.duplicated().sum()}\n")
    print(f"Unique Values : ")
    nunique_vals = df.nunique().reset_index().rename({"index" : "feature", 0 : "nunique_vals"}, axis = 1)
    display(nunique_vals)

    display(df.dtypes.reset_index().rename(columns = {"index" : "fetaure", 0 : "data type"}))

    return missing_data

In [4]:
def num_fillna(df, given_info, feature_median, col):
    required_data = given_info[(given_info.industry == df.industry) & (given_info.sector == df.sector)][col]
    if required_data.shape[0] == 0:
      required_data = given_info[(given_info.industry == df.industry)][col]
    elif required_data.shape[0] == 0:
      required_data = given_info[(given_info.sector == df.sector)][col]
    else:
         return feature_median
    return required_data.iloc[0]   

## reading data and finding basic info

In [5]:
train = pd.read_csv('/kaggle/input/financial-performance-prediction/train.csv')
test = pd.read_csv('/kaggle/input/financial-performance-prediction/test.csv')

In [6]:
train.head()

,Id,industry,sector,fullTimeEmployees,auditRisk,boardRisk,compensationRisk,shareHolderRightsRisk,overallRisk,trailingPE,...,Q10_TOTAL_STOCKHOLDERS_EQUITY,Q10_NET_INCOME,Q10_GROSS_PROFIT,Q10_COST_OF_REVENUES,Q10_REVENUES,Q10_OPERATING_INCOME,Q10_OPERATING_EXPENSES,Q10_EBITDA,Q10_DEPRECIATION_AND_AMORTIZATION,Q10_fiscal_year_end
0,196,Personal Services,Consumer Cyclical,1174.0,5.0,10.0,9.0,4.0,8.0,12.549223,...,2.405020e+08,NaN,2.317100e+07,5.431900e+07,4.073200e+07,2150000.0,5946000.0,1.722500e+07,15075000.0,0.0
1,1568,Building Products & Equipment,Industrials,3600.0,4.0,4.0,3.0,4.0,3.0,1222.000000,...,3.265380e+08,-71929000.0,4.105740e+08,6.926880e+08,1.103262e+09,-39287000.0,333229000.0,7.734500e+07,116632000.0,0.0
2,1218,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.066300e+10,603000000.0,1.649000e+09,4.587000e+09,6.236000e+09,663000000.0,214000000.0,1.435000e+09,772000000.0,0.0
3,23,Scientific & Technical Instruments,Technology,143.0,NaN,NaN,NaN,NaN,NaN,NaN,...,2.758000e+03,-9715.0,0.000000e+00,NaN,NaN,-9683.0,9683.0,-9.683000e+03,0.0,0.0
4,783,Drug Manufacturers - Specialty & Generic,Healthcare,36.0,NaN,NaN,NaN,NaN,NaN,1.231544,...,2.592400e+07,-4676000.0,1.205000e+07,0.000000e+00,1.205000e+07,-1594000.0,13644000.0,-1.594000e+06,0.0,0.0


In [7]:
test.head()

,Id,industry,sector,fullTimeEmployees,auditRisk,boardRisk,compensationRisk,shareHolderRightsRisk,overallRisk,trailingPE,...,Q10_TOTAL_STOCKHOLDERS_EQUITY,Q10_NET_INCOME,Q10_GROSS_PROFIT,Q10_COST_OF_REVENUES,Q10_REVENUES,Q10_OPERATING_INCOME,Q10_OPERATING_EXPENSES,Q10_EBITDA,Q10_DEPRECIATION_AND_AMORTIZATION,Q10_fiscal_year_end
0,480,Drug Manufacturers - Specialty & Generic,Healthcare,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,573,Drug Manufacturers - Specialty & Generic,Healthcare,7600.0,6.0,7.0,6.0,4.0,5.0,90.500000,...,3.230150e+08,NaN,191562000.0,301543000.0,493105000.0,45907000.0,90726000.0,100836000.0,54929000.0,0.0
2,1967,Software - Application,Technology,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,57,Marine Shipping,Industrials,5200.0,1.0,5.0,4.0,5.0,3.0,23.500002,...,3.085267e+09,-3375000.0,72793000.0,228158000.0,300951000.0,3164000.0,69629000.0,3164000.0,0.0,0.0
4,131,Metal Fabrication,Industrials,6700.0,1.0,4.0,4.0,6.0,3.0,20.748793,...,4.829000e+08,-7900000.0,85800000.0,606700000.0,692500000.0,31800000.0,54000000.0,31800000.0,0.0,0.0


In [8]:
missing_vals_train = basic_info(train)

shape of the date : 
	rows = 1624, columns = 212

missing values: 
	count = 19283


,feature,missing_val_count,missing_val_percentage
9,trailingPE,902,55.54
4,auditRisk,493,30.36
5,boardRisk,493,30.36
6,compensationRisk,493,30.36
7,shareHolderRightsRisk,493,30.36
...,...,...,...
59,Q2_TOTAL_ASSETS,1,0.06
65,Q2_TOTAL_LIABILITIES_AND_EQUITY,1,0.06
66,Q2_TOTAL_STOCKHOLDERS_EQUITY,1,0.06
96,Q4_TOTAL_LIABILITIES,1,0.06


duplicated records: 
	count = 0

Unique Values : 


,feature,nunique_vals
0,Id,1624
1,industry,113
2,sector,10
3,fullTimeEmployees,971
4,auditRisk,10
...,...,...
207,Q10_OPERATING_INCOME,1397
208,Q10_OPERATING_EXPENSES,1402
209,Q10_EBITDA,1398
210,Q10_DEPRECIATION_AND_AMORTIZATION,559


,fetaure,data type
0,Id,int64
1,industry,object
2,sector,object
3,fullTimeEmployees,float64
4,auditRisk,float64
...,...,...
207,Q10_OPERATING_INCOME,float64
208,Q10_OPERATING_EXPENSES,float64
209,Q10_EBITDA,float64
210,Q10_DEPRECIATION_AND_AMORTIZATION,float64


In [9]:
missing_vals_test = basic_info(test)

shape of the date : 
	rows = 406, columns = 203

missing values: 
	count = 5791


,feature,missing_val_count,missing_val_percentage
9,trailingPE,217,53.45
4,auditRisk,132,32.51
5,boardRisk,132,32.51
6,compensationRisk,132,32.51
7,shareHolderRightsRisk,132,32.51
...,...,...,...
107,Q5_TOTAL_LIABILITIES_AND_EQUITY,1,0.25
91,Q4_TOTAL_STOCKHOLDERS_EQUITY,1,0.25
74,Q3_TOTAL_STOCKHOLDERS_EQUITY,1,0.25
125,Q6_TOTAL_STOCKHOLDERS_EQUITY,1,0.25


duplicated records: 
	count = 0

Unique Values : 


,feature,nunique_vals
0,Id,406
1,industry,88
2,sector,10
3,fullTimeEmployees,311
4,auditRisk,10
...,...,...
198,Q10_OPERATING_INCOME,333
199,Q10_OPERATING_EXPENSES,334
200,Q10_EBITDA,330
201,Q10_DEPRECIATION_AND_AMORTIZATION,140


,fetaure,data type
0,Id,int64
1,industry,object
2,sector,object
3,fullTimeEmployees,float64
4,auditRisk,float64
...,...,...
198,Q10_OPERATING_INCOME,float64
199,Q10_OPERATING_EXPENSES,float64
200,Q10_EBITDA,float64
201,Q10_DEPRECIATION_AND_AMORTIZATION,float64


In [10]:
train.describe()

,Id,fullTimeEmployees,auditRisk,boardRisk,compensationRisk,shareHolderRightsRisk,overallRisk,trailingPE,forwardPE,floatShares,...,Q10_TOTAL_STOCKHOLDERS_EQUITY,Q10_NET_INCOME,Q10_GROSS_PROFIT,Q10_COST_OF_REVENUES,Q10_REVENUES,Q10_OPERATING_INCOME,Q10_OPERATING_EXPENSES,Q10_EBITDA,Q10_DEPRECIATION_AND_AMORTIZATION,Q10_fiscal_year_end
count,1624.000000,1544.000000,1131.000000,1131.000000,1131.000000,1131.000000,1131.000000,722.000000,1436.000000,1.598000e+03,...,1.449000e+03,1.305000e+03,1.423000e+03,1.321000e+03,1.325000e+03,1.427000e+03,1.430000e+03,1.421000e+03,1.419000e+03,1624.000000
mean,1012.267857,9345.583549,5.564103,5.654288,5.797524,5.582670,5.733864,inf,inf,2.260694e+08,...,2.165070e+09,7.483623e+07,4.547676e+08,8.925653e+08,1.355588e+09,1.195989e+08,3.839765e+08,6.886442e+07,-5.136951e+07,0.004926
std,585.242103,34273.510554,2.850260,2.857969,2.839455,2.769783,2.879112,NaN,NaN,3.579918e+09,...,8.504373e+09,5.173134e+08,2.242473e+09,4.225508e+09,5.776379e+09,6.788749e+08,1.677897e+09,1.775192e+09,1.728854e+09,0.070035
min,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000086,-2085.000000,1.034690e+05,...,-1.805800e+10,-3.944000e+09,-2.170800e+10,-1.880579e+09,-4.500000e+06,-4.475000e+09,-1.439000e+06,-5.090500e+10,-5.418200e+10,0.000000
25%,511.750000,141.000000,3.000000,3.000000,3.000000,3.000000,3.000000,12.575714,-2.843448,1.500746e+07,...,2.206300e+07,-1.088800e+07,1.615874e+06,1.202000e+06,9.223996e+06,-7.866176e+06,7.696162e+06,-1.061200e+07,0.000000e+00,0.000000
50%,1009.500000,878.000000,6.000000,6.000000,6.000000,6.000000,6.000000,21.599812,9.378077,4.103422e+07,...,2.365370e+08,-2.036240e+05,3.437100e+07,2.986900e+07,1.050790e+08,-5.481500e+04,3.821400e+07,-2.862240e+05,0.000000e+00,0.000000
75%,1519.250000,5040.750000,8.000000,8.000000,8.000000,8.000000,8.000000,37.157093,19.998548,1.062949e+08,...,9.976180e+08,2.411300e+07,2.109785e+08,3.116390e+08,6.035310e+08,4.251150e+07,1.740402e+08,3.760000e+07,0.000000e+00,0.000000
max,2029.000000,500000.000000,10.000000,10.000000,10.000000,10.000000,10.000000,inf,inf,1.421357e+11,...,1.622829e+11,9.497000e+09,2.957200e+10,6.617700e+10,9.240000e+10,1.137800e+10,2.919700e+10,2.656800e+10,2.551400e+10,1.000000


In [11]:
test.describe()

,Id,fullTimeEmployees,auditRisk,boardRisk,compensationRisk,shareHolderRightsRisk,overallRisk,trailingPE,forwardPE,floatShares,...,Q10_TOTAL_STOCKHOLDERS_EQUITY,Q10_NET_INCOME,Q10_GROSS_PROFIT,Q10_COST_OF_REVENUES,Q10_REVENUES,Q10_OPERATING_INCOME,Q10_OPERATING_EXPENSES,Q10_EBITDA,Q10_DEPRECIATION_AND_AMORTIZATION,Q10_fiscal_year_end
count,406.000000,384.000000,274.000000,274.000000,274.000000,274.000000,274.000000,189.000000,352.000000,4.020000e+02,...,3.500000e+02,3.180000e+02,3.380000e+02,3.210000e+02,3.210000e+02,3.360000e+02,3.390000e+02,3.360000e+02,3.320000e+02,406.000000
mean,1023.428571,8560.729167,5.540146,5.624088,5.923358,5.518248,5.759124,inf,inf,1.630020e+08,...,3.278283e+09,7.859257e+07,6.510392e+08,8.527320e+08,1.530937e+09,1.381918e+08,3.651953e+08,2.864123e+08,1.449516e+08,0.012315
std,590.432542,27004.049445,2.837516,2.835077,2.890803,2.820911,2.884912,NaN,NaN,5.599856e+08,...,1.163343e+10,4.729266e+08,3.871916e+09,3.293652e+09,6.249743e+09,6.360699e+08,1.096210e+09,3.092154e+09,2.686686e+09,0.110425
min,2.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000010,-844.500000,6.582440e+05,...,-2.309400e+09,-1.973000e+09,-1.181000e+09,-9.070000e+08,0.000000e+00,-1.558000e+09,1.100000e+03,-3.024000e+09,-4.564000e+09,0.000000
25%,482.750000,114.000000,3.000000,3.000000,3.250000,3.000000,3.000000,11.153847,-2.025302,1.564819e+07,...,1.320993e+07,-1.098983e+07,1.317500e+06,8.000000e+05,5.049000e+06,-8.700250e+06,7.633714e+06,-1.253950e+07,0.000000e+00,0.000000
50%,1045.000000,838.000000,6.000000,6.000000,6.000000,6.000000,6.000000,20.783356,9.615789,4.419933e+07,...,2.712990e+08,-2.210000e+05,3.233500e+07,2.504000e+07,8.771000e+07,2.755155e+05,4.639200e+07,-2.306930e+05,0.000000e+00,0.000000
75%,1538.750000,5550.000000,8.000000,8.000000,8.000000,8.000000,8.000000,36.758060,17.707155,1.115940e+08,...,1.357392e+09,3.253675e+07,2.810150e+08,3.842000e+08,6.990540e+08,6.249350e+07,1.815205e+08,5.055325e+07,0.000000e+00,0.000000
max,2026.000000,339341.000000,10.000000,10.000000,10.000000,10.000000,10.000000,inf,inf,8.945318e+09,...,1.274871e+11,5.105000e+09,6.262400e+10,4.089400e+10,7.019600e+10,6.739000e+09,1.022300e+10,5.240100e+10,4.566200e+10,1.000000


- there are some outlers,
- and there are some inf values

In [12]:
target_cols = [col for col in train.columns if col not in test.columns]

In [13]:
train.replace([np.inf, -np.inf], np.nan, inplace=True)
test.replace([np.inf, -np.inf], np.nan, inplace=True)

## Data Preprocessing

In [14]:
meta_data_cols = [col for col in train.columns if train[col].dtype != "O" and col[0] != "Q"]
meta_data = train[meta_data_cols]
all_nan_metadata = train[meta_data.isna().all(axis=1)]

train = train.drop(all_nan_metadata.index, axis = 0)
train.shape

(1624, 212)

In [15]:
missing_cat_cols = []
for col in missing_vals_train.feature:
    if train[col].dtype == "O":
        missing_cat_cols.append(col)
        display(missing_vals_train[missing_vals_train.feature == col])

,feature,missing_val_count,missing_val_percentage
1,industry,22,1.35


,feature,missing_val_count,missing_val_percentage
20,recommendationKey,20,1.23


,feature,missing_val_count,missing_val_percentage
31,financialCurrency,20,1.23


In [16]:
for col in [col for col in train.columns if train[col].dtype == "O"]:        
    train[col] = train[col].fillna("Unknown")
    test[col] = test[col].fillna("Unknown")

In [17]:
num_cols = train.columns[train.isna().sum() > 0]
given_info = train.groupby(by = ["industry", "sector"])[num_cols].apply("median").reset_index()
# given_info.head(10)

In [18]:
quarters = ["Q"+str(i) for i in range (0,11)]

quater_wise_cols = {}
for col in [col for col in train.columns if train[col].dtype != "O"]:
    if col[0] == "Q":
        if col[1] + col[2] != "10" and col[1] != "0":
            if col[0:2] in quater_wise_cols.keys():
                quater_wise_cols[col[0:2]].append(col)
            else:
                quater_wise_cols[col[0:2]] = [col] 
        else:
            if "Q10" in col:
                if col[0:3] in quater_wise_cols.keys():
                    quater_wise_cols[col[0:3]].append(col)
                else:
                    quater_wise_cols[col[0:3]] = [col]
            else:
                if col[0:2] in quater_wise_cols.keys():
                    quater_wise_cols[col[0:2]].append(col)
                else:
                    quater_wise_cols[col[0:2]] = [col]  

for quarter in quater_wise_cols.keys():
    print(f"{quarter} : {len(quater_wise_cols[quarter])}")

Q0 : 10
Q1 : 17
Q2 : 17
Q3 : 17
Q4 : 17
Q5 : 17
Q6 : 17
Q7 : 17
Q8 : 17
Q9 : 17
Q10 : 17


In [19]:
neighbour_quarters = {
        "Q2" : ["Q1", "Q3"],
        "Q3" : ["Q2", "Q4"],
        "Q4" : ["Q3", "Q5"],
        "Q5" : ["Q4", "Q6"],
        "Q6" : ["Q5", "Q7"],
        "Q7" : ["Q6", "Q8"],
        "Q8" : ["Q7", "Q8"],
        "Q9" : ["Q8", "Q9"]
}


In [20]:
for col in [col for col in train.columns if train[col].dtype != "O" and col[0] == "Q"]:
    if "Q0" not in col and "Q10" not in col and "Q1" not in col:
        seeing_quarter = col[0:2]
        neighbour_quarter = neighbour_quarters[seeing_quarter]
        feature = col[2:]

        left_neighbour = train.copy()
        right_neighbour = train.copy()

        train_median_left = left_neighbour[neighbour_quarter[0] + feature].median()
        train_median_right = right_neighbour[neighbour_quarter[1] + feature].median()


        left_neighbour[neighbour_quarter[0] + feature] = left_neighbour[neighbour_quarter[0] + feature].fillna(train_median_left)
        right_neighbour[neighbour_quarter[1] + feature] = right_neighbour[neighbour_quarter[1] + feature].fillna(train_median_right)
    
        left_neighbour = left_neighbour.loc[train[col].isna(), neighbour_quarter[0] + feature]
        right_neighbour = right_neighbour.loc[train[col].isna(), neighbour_quarter[1] + feature]

        train.loc[train[col].isna(), col] = (right_neighbour + 2*(left_neighbour))/3

        left_neighbour = test.copy()
        right_neighbour = test.copy()

        left_neighbour[neighbour_quarter[0] + feature] = left_neighbour[neighbour_quarter[0] + feature].fillna(train_median_left)
        right_neighbour[neighbour_quarter[1] + feature] = right_neighbour[neighbour_quarter[1] + feature].fillna(train_median_right)
    
        left_neighbour = left_neighbour.loc[test[col].isna(), neighbour_quarter[0] + feature]
        right_neighbour = right_neighbour.loc[test[col].isna(), neighbour_quarter[1] + feature]

        test.loc[test[col].isna(), col] = (right_neighbour + (2 *left_neighbour))/3
       
        

In [21]:
num_cols = [col for col in train.columns if train[col].dtype != "O"]

for col in num_cols:
    if train[col].isna().sum() > 0:
        print(f"{col} = {train[col].isna().sum()}")

fullTimeEmployees = 80
auditRisk = 493
boardRisk = 493
compensationRisk = 493
shareHolderRightsRisk = 493
overallRisk = 493
trailingPE = 909
forwardPE = 194
floatShares = 26
sharesOutstanding = 23
trailingEps = 47
forwardEps = 194
targetHighPrice = 307
targetLowPrice = 307
targetMeanPrice = 307
targetMedianPrice = 307
recommendationMean = 224
numberOfAnalystOpinions = 307
totalCash = 27
totalCashPerShare = 57
ebitda = 54
totalDebt = 64
totalRevenue = 50
revenuePerShare = 63
freeCashflow = 33
operatingCashflow = 29
revenueGrowth = 112
Q1_TOTAL_ASSETS = 1
Q1_TOTAL_CURRENT_ASSETS = 10
Q1_TOTAL_NONCURRENT_ASSETS = 11
Q1_TOTAL_CURRENT_LIABILITIES = 13
Q1_TOTAL_NONCURRENT_LIABILITIES = 13
Q1_NET_INCOME = 230
Q1_GROSS_PROFIT = 7
Q1_COST_OF_REVENUES = 19
Q1_REVENUES = 19
Q1_OPERATING_INCOME = 7
Q1_OPERATING_EXPENSES = 8
Q1_EBITDA = 8
Q1_DEPRECIATION_AND_AMORTIZATION = 8
Q10_TOTAL_ASSETS = 178
Q10_TOTAL_CURRENT_ASSETS = 186
Q10_TOTAL_NONCURRENT_ASSETS = 189
Q10_TOTAL_LIABILITIES = 175
Q10_TOTAL

In [22]:
for col in train.columns[train.isna().sum() > 0]:
    feature_median = train[col].median()
    train[col] = train[col].fillna(feature_median)
    test[col] = test[col].fillna(feature_median)

In [23]:
print(f"misisng values in train data = {train.columns[train.isna().sum() > 0]}")
print(f"misisng values in test data = {test.columns[test.isna().sum() > 0]}")

misisng values in train data = Index([], dtype='object')
misisng values in test data = Index(['Q1_TOTAL_LIABILITIES', 'Q1_TOTAL_LIABILITIES_AND_EQUITY',
       'Q1_TOTAL_STOCKHOLDERS_EQUITY'],
      dtype='object')


In [24]:
for col in test.columns[test.isna().sum() > 0]:
    feature_median = train[col].median()
    # train[col] = train[col].fillna(feature_median)
    test[col] = test[col].fillna(feature_median)

In [25]:
print(f"misisng values in train data = {train.columns[train.isna().sum() > 0]}")
print(f"misisng values in test data = {test.columns[test.isna().sum() > 0]}")

misisng values in train data = Index([], dtype='object')
misisng values in test data = Index([], dtype='object')


In [26]:
cat_cols = [col for col in train.columns if train[col].dtype == "O" or train[col].nunique() <= 30]

## Model

In [27]:
X = train.drop(target_cols, axis = 1).copy()
y = train[target_cols].copy()

In [ ]:
xgb = XGBRegressor(n_jobs = -1)
rf = RandomForestRegressor(n_jobs= -1)
train_preds = {}
test_preds = {}
print('Cross validation R2 scores for each target:\n')
cross_val_score_results = {}
for target in y:

    encoder = TargetEncoder(cols = cat_cols)
    encoder.fit(X, y[target])
    X_target = encoder.transform(X)
    test_target = encoder.transform(test)

    scaler = StandardScaler()
    scaler.fit(X_target)

    X_train_sca = scaler.transform(X_target)
    test_target = scaler.transform(test_target)

    if target in ["Q0_TOTAL_LIABILITIES", "Q0_EBITDA" ,"Q0_GROSS_PROFIT", "Q0_COST_OF_REVENUES", "Q0_OPERATING_EXPENSES"]:
        score = np.round(np.mean(cross_val_score(rf, X_train_sca, y[target], cv=3, scoring='r2')),2)
    else:
        score = np.round(np.mean(cross_val_score(xgb, X_train_sca, y[target], cv=3, scoring='r2')),2)
    cross_val_score_results[target] = score
    
    if target in ["Q0_TOTAL_LIABILITIES", "Q0_EBITDA" ,"Q0_GROSS_PROFIT", "Q0_COST_OF_REVENUES", "Q0_OPERATING_EXPENSES"]:
        rf = RandomForestRegressor(n_jobs = -1)
        rf.fit(X_train_sca, y[target])
        train_preds[target] = rf.predict(X_train_sca)
        test_preds[target] = rf.predict(test_target)
    else:
        xgb = XGBRegressor(n_jobs = -1)
        xgb.fit(X_train_sca, y[target])
        train_preds[target] = xgb.predict(X_train_sca)
        test_preds[target] = xgb.predict(test_target)
    print(f'{target} r2_score -> {score}')
    
print(f'\nMean R2 score across all targets: {np.mean(list(cross_val_score_results.values()))}')

Cross validation R2 scores for each target:

Q0_TOTAL_ASSETS r2_score -> 0.75
Q0_TOTAL_LIABILITIES r2_score -> 0.67
Q0_TOTAL_STOCKHOLDERS_EQUITY r2_score -> 0.9


In [ ]:
submission = pd.read_csv("/kaggle/input/financial-performance-prediction/sample_submission.csv")

for target in test_preds:
    submission[target] = test_preds[target]    

submission.to_csv('submission.csv', index = False)